In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('./merged_twitter.csv')

In [3]:
df.head(15)

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative
5,http://www.dothebouncy.com/smf - some shameles...,neutral
6,fun,positive
7,Soooo high,neutral
8,Both of you,neutral
9,Wow... u just became cooler.,positive


In [4]:
len(df)

190461

In [5]:
print(df.isnull().sum())


selected_text    68
sentiment         7
dtype: int64


In [6]:
df_dropped = df.dropna(inplace = True)

In [7]:
print(df.isnull().sum())


selected_text    0
sentiment        0
dtype: int64


In [8]:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [9]:
df.groupby('sentiment').nunique()

,selected_text
sentiment,
negative,40792
neutral,64190
positive,76393


In [10]:
df = df[['selected_text', 'sentiment']]
print(df.head(10))
print(df.tail(10))

                                       selected_text sentiment
0                I`d have responded, if I were going   neutral
1                                           Sooo SAD  negative
2                                        bullying me  negative
3                                     leave me alone  negative
4                                      Sons of ****,  negative
5  http://www.dothebouncy.com/smf - some shameles...   neutral
6                                                fun  positive
7                                         Soooo high   neutral
8                                        Both of you   neutral
9                       Wow... u just became cooler.  positive
                                            selected_text sentiment
190451  raga knows dat wont spared modi courts cheatin...  positive
190452  congress veteran sudhakar reddy joins bjp meet...   neutral
190453  engine growth modi unveils indias first 12000 ...  positive
190454  modi promised 2014 lok sabh

In [11]:
df['selected_text'].isnull().sum()

0

In [12]:
df['selected_text'].fillna("No content", inplace = True)
print("Done")

Done


In [13]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

def preprocess_text(texts):
    preprocessed_texts = []
    stop_words = set(stopwords.words('english'))
    
    for text in texts:
        # Remove HTML tags
        text = BeautifulSoup(text, 'html.parser').get_text()

        # Remove URLs
        text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

        # Tokenize text
        tokenizer = TweetTokenizer()
        tokens = tokenizer.tokenize(text)

        # Remove emails
        tokens = [token for token in tokens if not re.match(r'\S+@\S+', token)]

        # Remove new lines characters
        tokens = [token for token in tokens if token != '\n']

        # Remove distracting single quotes
        tokens = [token.replace("'", "") for token in tokens]

        # Remove all punctuation signs and stopwords
        tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

        # Detokenize text
        preprocessed_texts.append(' '.join(tokens))

    return np.array(preprocessed_texts)
# Preprocess 'tweet' column
df['selected_text'] = preprocess_text(df['selected_text'])

print(df)

df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1, 'neutral': 2})

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['selected_text'])

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Print the first most frequent words


sequences = tokenizer.texts_to_sequences(df['selected_text'])
X = pad_sequences(sequences, maxlen=max_len)

# Convert sentiment column to numpy array
y = np.array(df['sentiment'])

print(X)


C:\Users\Dhananjay\AppData\Local\Temp\ipykernel_4608\2241134619.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


                                            selected_text sentiment
0                                         responded going   neutral
1                                                sooo sad  negative
2                                                bullying  negative
3                                             leave alone  negative
4                                                    sons  negative
...                                                   ...       ...
190456  456 crores paid neerav modi recovered congress...  negative
190457  dear rss terrorist payal gawar modi killing 10...  negative
190458                       cover interaction forum left   neutral
190459  big project came india modi dream project happ...   neutral
190460  ever listen like gurukul discipline maintained...  positive

[190386 rows x 2 columns]
[[   0    0    0 ...    0 4247   74]
 [   0    0    0 ...    0 3977  442]
 [   0    0    0 ...    0    0    0]
 ...
 [   0    0    0 ... 1573 2599  295]
 [  

In [14]:
print(y)

[2 0 0 ... 2 2 1]


In [15]:
from sklearn.model_selection import train_test_split

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Train set size:", len(X_train))
print("Test set size:", len(X_test))
print("Train labels size:", len(y_train))
print("Test labels size:", len(y_test))

Train set size: 152308
Test set size: 38078
Train labels size: 152308
Test labels size: 38078


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Define your neural network model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128))
model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile your model with a lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train your model
history = model.fit(X_train, y_train, batch_size=128, epochs=5, validation_split=0.2, class_weight=class_weights_dict)

# Save the model as .h5 file
model.save("best_model2.h5")

# Evaluate your model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/5
 51/952 [>.............................] - ETA: 55:14 - loss: 1.0647 - accuracy: 0.4412

KeyboardInterrupt: 

In [ ]:
# Function to preprocess a single text
def preprocess_single_text(text):
    stop_words = set(stopwords.words('english'))

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove URLs
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

    # Tokenize text
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)

    # Remove emails
    tokens = [token for token in tokens if not re.match(r'\S+@\S+', token)]

    # Remove new lines characters
    tokens = [token for token in tokens if token != '\n']

    # Remove distracting single quotes
    tokens = [token.replace("'", "") for token in tokens]

    # Remove all punctuation signs and stopwords
    tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

    # Detokenize text
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Preprocess a single text
text_to_classify = "I was really disappointed with this product. It didn't live up to the hype at all. The quality was subpar, and it broke within a week of use. Definitely not worth the money."
preprocessed_text = preprocess_single_text(text_to_classify)

# Tokenize and pad the preprocessed text
sequence = tokenizer.texts_to_sequences([preprocessed_text])
padded_sequence = pad_sequences(sequence, maxlen=max_len)

# Predict the sentiment class
predicted_class = np.argmax(model.predict(padded_sequence), axis=-1)[0]

# Map predicted class to sentiment label
sentiment_mapping = {0: 'negative', 1: 'positive', 2: 'neutral'}
predicted_sentiment = sentiment_mapping[predicted_class]

print("Predicted Sentiment:", predicted_sentiment)


In [ ]:
print("TensorFlow version:", tf.__version__)